In [1]:
from xverif.wrappers import ensure_dataarray
from xverif.datasets import create_ensemble_forecast_dataset, create_spatial2d_dataset
from xverif.metrics.deterministic.continuous_vectorized import (
    get_available_metrics,
    get_stacking_dict,
    get_metrics
)

sample_dims = ["x", "y"]
sample_dims = sample_dims
metrics = get_available_metrics()

obs = create_spatial2d_dataset(1000)
pred = create_ensemble_forecast_dataset(1000) 

/home/ghiggi/Python_Packages/xverif/xverif/datasets.py:71: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da = xr.DataArray(data, coords=coords, dims=dims, name=name)
/home/ghiggi/Python_Packages/xverif/xverif/datasets.py:71: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constru

In [2]:
# Chunk finely on auxiliary dimension if going out of memory !
pred = pred.chunk({"x": -1, "y": -1, 
                   # "realization": 1, 
                   "leadtime": 1})
obs = obs.chunk({"x": -1, "y": -1})

pred.chunks

Frozen({'time': (1000,), 'x': (10,), 'y': (10,), 'realization': (24,), 'leadtime': (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)})

In [3]:
pred['var0']

<xarray.DataArray 'var0' (time: 1000, x: 10, y: 10, realization: 24,
                          leadtime: 48)>
dask.array<xarray-var0, shape=(1000, 10, 10, 24, 48), dtype=float64, chunksize=(1000, 10, 10, 24, 1), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 1992-07-22 1992-07-23 ... 1995-04-17
  * x            (x) int64 0 1 2 3 4 5 6 7 8 9
  * y            (y) int64 0 1 2 3 4 5 6 7 8 9
  * realization  (realization) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
  * leadtime     (leadtime) int64 0 1 2 3 4 5 6 7 8 ... 40 41 42 43 44 45 46 47

In [4]:
# Reshaping operations
pred = ensure_dataarray(pred)
obs = ensure_dataarray(obs)

In [5]:
pred

<xarray.DataArray (variable: 3, time: 1000, x: 10, y: 10, realization: 24,
                   leadtime: 48)>
dask.array<stack, shape=(3, 1000, 10, 10, 24, 48), dtype=float64, chunksize=(1, 1000, 10, 10, 24, 1), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 1992-07-22 1992-07-23 ... 1995-04-17
  * x            (x) int64 0 1 2 3 4 5 6 7 8 9
  * y            (y) int64 0 1 2 3 4 5 6 7 8 9
  * realization  (realization) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
  * leadtime     (leadtime) int64 0 1 2 3 4 5 6 7 8 ... 40 41 42 43 44 45 46 47
  * variable     (variable) object 'var0' 'var1' 'var2'

In [6]:
# Broadcast obs to pred
# - Creates a view, not a copy !
obs = obs.broadcast_like(pred)
# obs_broadcasted['var0'].data.flags # view (Both contiguous are False)
# obs_broadcasted['var0'].data.base  # view (Return array and not None)

# Ensure obs same chunks of pred on auxiliary dims
# TODO: now makes same as pred --> TO IMPROVE
obs = obs.chunk(pred.chunks)

In [7]:
obs

<xarray.DataArray (variable: 3, time: 1000, x: 10, y: 10, realization: 24,
                   leadtime: 48)>
dask.array<rechunk-merge, shape=(3, 1000, 10, 10, 24, 48), dtype=float64, chunksize=(1, 1000, 10, 10, 24, 1), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 1992-07-22 1992-07-23 ... 1995-04-17
  * x            (x) int64 0 1 2 3 4 5 6 7 8 9
  * y            (y) int64 0 1 2 3 4 5 6 7 8 9
  * variable     (variable) object 'var0' 'var1' 'var2'
  * realization  (realization) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
  * leadtime     (leadtime) int64 0 1 2 3 4 5 6 7 8 ... 40 41 42 43 44 45 46 47

In [8]:
# Stack pred and obs to have 2D dimensions (aux, sample)
# - This operation doubles the memory
stacking_dict = get_stacking_dict(pred, sample_dims=sample_dims)
pred = pred.stack(stacking_dict)
obs = obs.stack(stacking_dict)

/home/ghiggi/.cache/pypoetry/virtualenvs/xverif-7K50T8uA-py3.10/lib/python3.10/site-packages/xarray/core/variable.py:1745: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._stack_once(dims, new_dim)
/home/ghiggi/.cache/pypoetry/virtualenvs/xverif-7K50T8uA-py3.10/lib/python3.10/site-packages/xarray/core/variable.py:1745: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': F

In [9]:
pred

<xarray.DataArray (aux: 3456000, sample: 100)>
dask.array<reshape, shape=(3456000, 100), dtype=float64, chunksize=(1152000, 100), chunktype=numpy.ndarray>
Coordinates:
  * aux          (aux) object MultiIndex
  * variable     (aux) object 'var0' 'var0' 'var0' ... 'var2' 'var2' 'var2'
  * realization  (aux) int64 0 0 0 0 0 0 0 0 0 0 ... 23 23 23 23 23 23 23 23 23
  * leadtime     (aux) int64 0 0 0 0 0 0 0 0 0 0 ... 47 47 47 47 47 47 47 47 47
  * time         (aux) datetime64[ns] 1992-07-22 1992-07-23 ... 1995-04-17
  * sample       (sample) object MultiIndex
  * x            (sample) int64 0 0 0 0 0 0 0 0 0 0 1 ... 8 9 9 9 9 9 9 9 9 9 9
  * y            (sample) int64 0 1 2 3 4 5 6 7 8 9 0 ... 9 0 1 2 3 4 5 6 7 8 9

In [20]:
# Visualize dask operations here 
# TODO

# Compute metrics on dask array 
a = get_metrics(pred.data, obs.data, metrics) # 16 seconds 


# Visualize tasks 
# a.visualize

In [21]:
a

dask.array<stack, shape=(3456000, 64), dtype=float64, chunksize=(1152000, 1), chunktype=numpy.ndarray>

In [14]:
a = get_metrics(pred.data, obs.data, metrics=["spearman_R"])  

In [15]:
a

dask.array<stack, shape=(3456000, 1), dtype=float64, chunksize=(1152000, 1), chunktype=numpy.ndarray>

In [17]:
a = get_metrics(pred.data, obs.data, metrics=["MAE"])  

In [18]:
a

dask.array<stack, shape=(3456000, 1), dtype=float64, chunksize=(1152000, 1), chunktype=numpy.ndarray>

In [19]:
 166*64

10624